In [1]:
import xmlrpc.client
import pandas as pd

# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# 1) Traer data personal activo 
# ========================================
users = models.execute_kw(
    db, uid, password,
    "hr.employee", "search_read",
    [[ ["active", "=", True]]],
    {"fields": ["ubicacion_trabajo","department_id","display_name","work_email"]}
)
df = pd.DataFrame(users)

df["department_name"] = df["department_id"].apply(lambda x: x[1] if isinstance(x, list) else None)

# Separar la ruta por slash " / "
df[["empresa", "area", "cargo"]] = df["department_name"].str.split("/", expand=True)

# Limpiar espacios
df["area"] = df["area"].str.strip()
df["cargo"] = df["cargo"].str.strip()


In [2]:
df = df.drop(columns=["empresa","department_name","department_id"])

In [3]:
df.sample(10)

,work_email,ubicacion_trabajo,id,display_name,area,cargo
95,False,"[210, [08001] Atlántico / Barranquilla]",745,SAID DAVID VARGAS LOPEZ,GERENCIA GENERAL,COMERCIAL
109,False,"[563, [25214] Cundinamarca / Cota]",649,YEISON VILLANUEVA MELO,PRODUCCION,None
108,False,"[563, [25214] Cundinamarca / Cota]",752,YEIMY YULIANA JIMENEZ ANACONA,APRENDICES SENA,None
45,azubieta@donsson.com,"[563, [25214] Cundinamarca / Cota]",319,HUGO ALEJANDRO ZUBIETA FERNANDEZ,GERENCIA GENERAL,COMERCIAL
37,False,"[563, [25214] Cundinamarca / Cota]",672,ESTEBAN DAVID DIAZ SEGUANE,PRODUCCION,None
9,produccion@donsson.com,"[563, [25214] Cundinamarca / Cota]",279,BLANCA DORA CUBIDES SANCHEZ,PRODUCCION,None
92,patuk1967@gmail.com,"[1130, [76892] Valle del Cauca / Yumbo]",597,PATRICIA NUÑEZ VALENCIA,GERENCIA GENERAL,COMERCIAL
72,False,"[210, [08001] Atlántico / Barranquilla]",683,KENER DAVID SANTAMARIA BLANCO,GERENCIA GENERAL,COMERCIAL
13,False,"[563, [25214] Cundinamarca / Cota]",749,EDUARDO ALONSO CARDENAS MURILLO,PRODUCCION,None
30,False,"[1298, [68307] Santander / Girón]",684,EDINSON JAVIER GARCIA MURILLO,GERENCIA GENERAL,COMERCIAL


In [4]:
df_def = df[df["cargo"]=="COMERCIAL"]

In [5]:
df_def.sample(5)

,work_email,ubicacion_trabajo,id,display_name,area,cargo
28,False,"[85, [05001] Antioquia / Medellín]",697,DIEGO ANDRES CARDONA VILLA,GERENCIA GENERAL,COMERCIAL
7,False,"[233, [11001] Bogotá / Bogotá, D.C.]",698,AURELYSMAR REA SILVA,GERENCIA GENERAL,COMERCIAL
49,ventasandres@donsson.com,"[563, [25214] Cundinamarca / Cota]",268,JAIME ANDRES CABALLERO CABALLERO,GERENCIA GENERAL,COMERCIAL
62,False,"[563, [25214] Cundinamarca / Cota]",538,JUAN DAVID SOGAMOSO TIRADO,GERENCIA GENERAL,COMERCIAL
33,False,"[563, [25214] Cundinamarca / Cota]",278,EDWIN FERNANDO COLLAZOS BARRETO,GERENCIA GENERAL,COMERCIAL


In [6]:
import re

# Extraer el texto (posición 1 de la lista)
df_def["ubi"] = df_def["ubicacion_trabajo"].apply(
    lambda x: x[1] if isinstance(x, list) and len(x) > 1 else None
)

# Quitar los corchetes con número: [25214]
df_def["ubi"] = df_def["ubi"].apply(
    lambda x: re.sub(r"\[\d+\]\s*", "", x) if isinstance(x, str) else None
)

# Separar asegurando siempre 2 columnas
df_def[["departamento", "ciudad"]] = (
    df_def["ubi"]
    .str.split("/", n=1, expand=True)
    .reindex(columns=[0, 1])  # <-- garantiza que existan 2 columnas
)

df_def = df_def.drop(columns=["ubicacion_trabajo","ubi"])



/tmp/ipykernel_3210/3679110601.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_def["ubi"] = df_def["ubicacion_trabajo"].apply(
/tmp/ipykernel_3210/3679110601.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_def["ubi"] = df_def["ubi"].apply(
/tmp/ipykernel_3210/3679110601.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [7]:
df_def.sample(10)

,work_email,id,display_name,area,cargo,departamento,ciudad
101,ventasalejandro@donsson.com,286,VICTOR ALEJANDRO GONZALEZ RODRIGUEZ,GERENCIA GENERAL,COMERCIAL,Cundinamarca,Cota
96,False,551,SANDRA PATRICIA RAMIREZ CORREDOR,GERENCIA GENERAL,COMERCIAL,Santander,Bucaramanga
22,david.fernandez@donsson.com,671,DAVID DE JESUS FERNANDEZ TETTE,GERENCIA GENERAL,COMERCIAL,Atlántico,Barranquilla
89,neyder_1227@hotmail.com,658,NEYDER ENRIQUE CRESPO YEPES,GERENCIA GENERAL,COMERCIAL,Atlántico,Barranquilla
47,ventasisaac@donsson.com,311,ISAAC SANCHEZ MONTAÑO,GERENCIA GENERAL,COMERCIAL,Cundinamarca,Cota
38,False,679,EVERTH DE JESUS QUICENO JIMENEZ,GERENCIA GENERAL,COMERCIAL,Antioquia,Itagüí
14,07cpandres@gmail.com,742,CARLOS ANDRES CABUYALES PRADO,GERENCIA GENERAL,COMERCIAL,Valle del Cauca,Yumbo
3,andres.corrales@donsson.com,472,ANDRES EMILIO CORRALES CORBY,GERENCIA GENERAL,COMERCIAL,Cundinamarca,Cota
55,Fabian.blanco@donsson.com,705,JEYSSON FABIAN BLANCO RODRIGUEZ,GERENCIA GENERAL,COMERCIAL,Santander,Girón
32,ventasedison@donsson.com,275,EDISSON MIGUEL CIFUENTES OSORIO,GERENCIA GENERAL,COMERCIAL,Santander,Bucaramanga


In [8]:
df_def.to_excel("vendedores.xlsx")